In [6]:
import SimpleITK as sitk
import numpy as np
import mri_helpers as mh

interactive(children=(IntSlider(value=14, description='z', max=29), Output()), _dom_classes=('widget-interact'…

In [36]:
def resample_to(mha_in, mha_ref, is_label):
    ''' Resample an image from an input space (in_size, in_spacing, in_origin) to a reference space.'''
    # Adapted from https://gist.github.com/zivy/79d7ee0490faee1156c1277a78e4a4c4
    b2b = sitk.ReadImage(mha_in)
    ref = sitk.ReadImage(mha_ref)
    s_size, s_spacing, s_origin = b2b.GetSize(), b2b.GetSpacing(), b2b.GetOrigin()
    t_size, t_spacing, t_origin = ref.GetSize(), ref.GetSpacing(), ref.GetOrigin()

    # Always use the TransformContinuousIndexToPhysicalPoint to compute an indexed point's physical coordinates as 
    # this takes into account size, spacing and direction cosines. For the vast majority of images the direction 
    # cosines are the identity matrix, but when this isn't the case simply multiplying the central index by the 
    # spacing will not yield the correct coordinates resulting in a long debugging session. 
    t_center = np.array(ref.TransformContinuousIndexToPhysicalPoint(np.array(ref.GetSize())/2.0))

    dimension = b2b.GetDimension()

    # Transform which maps from the reference_image to the current img with the translation mapping the image
    # origins to each other.
    transform = sitk.AffineTransform(dimension)
    transform.SetMatrix(b2b.GetDirection())
    transform.SetTranslation(np.array(b2b.GetOrigin()) - t_origin)
    # Modify the transformation to align the centers of the original and reference image instead of their origins.
    centering_transform = sitk.TranslationTransform(dimension)
    img_center = np.array(b2b.TransformContinuousIndexToPhysicalPoint(np.array(b2b.GetSize())/2.0))
    centering_transform.SetOffset(np.array(transform.GetInverse().TransformPoint(img_center) - t_center))
    centered_transform = sitk.Transform(transform)
    centered_transform.AddTransform(centering_transform)
    # Using the linear interpolator as these are intensity images, if there is a need to resample a ground truth 
    # segmentation then the segmentation image should be resampled using the NearestNeighbor interpolator so that 
    # no new labels are introduced.
    interpolator = sitk.sitkLinear if not is_label else sitk.sitkNearestNeighbor
    return sitk.Resample(b2b, ref, centered_transform, interpolator, 0.0)
    
    

In [42]:
res = resample_to('MR_T1_TSE.mha', 'VSD.Brain.XX.O.MR_T1.54609.mha', is_label=False)

In [41]:
mh.myshow(res)

interactive(children=(IntSlider(value=77, description='z', max=154), Output()), _dom_classes=('widget-interact…

In [16]:
ref.GetPixelIDValue()

2

In [28]:
t_size, t_spacing, t_origin
mh.myshow(resampled)
    r_size, r_spacing, r_origin = resampled.GetSize(), resampled.GetSpacing(), resampled.GetOrigin()

((240, 240, 155), (1.0, 1.0, 1.0), (0.0, -239.0, 0.0))

In [30]:
r_size, r_spacing, r_origin

((240, 240, 155), (1.0, 1.0, 1.0), (0.0, -239.0, 0.0))